In [604]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI

%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [565]:
np.load("./bkup/best_subjects_search_results/subject_6_shallow_conv_net_7b71b3b6-52f7-435c-be85-2cb99e793f98.npy", allow_pickle=True).item().user_attrs['trial_data']

{'channels_selected': array(['P3', 'C3', 'F4', 'Fp1', 'T3', 'O1', 'F7', 'A2', 'T6'], dtype='<U3'),
 'model': '{"module": "keras.src.models.functional", "class_name": "Functional", "config": {"name": "functional_2431", "trainable": true, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 9, 513, 1], "dtype": "float32", "sparse": false, "name": "input_layer_1143"}, "registered_name": null, "name": "input_layer_1143", "inbound_nodes": []}, {"module": "keras.layers", "class_name": "Conv2D", "config": {"name": "conv2d_2250", "trainable": true, "dtype": "float32", "filters": 190, "kernel_size": [1, 33], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "clas

In [605]:
model_optimizer = ModelOptimizer(
    dataset=FatigueMI(),
    model_name="shallow_conv_net"
)

In [610]:
study = model_optimizer.search_best_model(
    subjects=[7],
    # channels=[],
    max_iter = 50,
    max_epochs=200,
    max_stag_count=25,
    rounds=1,
    replace_previous_study_for_subjects=True
)

/home/arazzz/Documents/GitHub/moabb_model_optimization/model_optimization_tests/model_optim/model_optimizer.py:377: ExperimentalWarning: NSGAIIISampler is experimental (supported from v3.2.0). The interface can change in the future.
  study.enqueue_trial(


  0%|          | 0/50 [00:00<?, ?it/s]

Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns
Epoch 1/200


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [607]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)
# model_optimizer.get_study_metrics(study).sort_values(by="test_acc", ascending=False)

,train_acc,test_acc,val_acc,train_val_diff,scores,channels_selected
23,0.867647,0.545455,0.722222,0.145425,1.809025,"[P3, F4, C4, Fp2, T3, T5, F8]"
24,0.970588,0.409091,0.555556,0.415033,1.869772,"[C3, F4, P4, Pz, T3, O1, A2, T6]"
22,0.970588,0.409091,0.722222,0.248366,1.899586,"[P3, C3, Pz, T3, F7, F8, A2, T4]"
25,1.000000,0.636364,0.888889,0.111111,1.932096,"[C3, Fz, F4, C4, P4, Pz, T5, O1, A2, T6]"
21,0.852941,0.590909,0.777778,0.075163,1.940876,"[P3, C3, Cz, T3, O1, F7, A2]"
29,1.000000,0.636364,0.555556,0.444444,2.015513,"[C3, F3, F4, P4, Cz, Pz, Fp1, T3, F8, A2]"
7,1.000000,0.590909,0.611111,0.388889,2.220413,"[C3, F4, P4, Cz, Pz, T3, T5, F7, A2, T6, T4]"
40,0.955882,0.500000,0.666667,0.289216,2.223789,"[P3, F3, Fz, P4, Cz, F7, F8, T4]"
8,0.941176,0.363636,0.611111,0.330065,2.244394,"[C3, F3, Fz, C4, P4, Fp2, O1, O2, F8, A2, T6]"
41,0.941176,0.409091,0.666667,0.274510,2.264134,"[P3, C3, F3, F4, C4, Pz, O1, O2, F8, T4]"


In [608]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': 128,
│   'batch_size': 128,
│   'pool_size_d2': 85,
│   'strides_d2': 18,
│   'conv_filters_d2': 24,
│   'conv2d_1_units': 100,
│   'conv2d_2_units': 60,
│   'l2_reg_1': 0.18150179301121425,
│   'l2_reg_2': 0.6480618097076263,
│   'l2_reg_3': 0.16286474156780992,
│   'dropout_rate': 0.9
}

test_accuracy = 0.5454545617103577

val_accuracy = 0.7222222089767456

channels_selected = ['P3' 'F4' 'C4' 'Fp2' 'T3' 'T5' 'F8']

In [609]:
model_optimizer.clean_up(
    action="remove_all_but_best_trial_data",
    subjects=[7],
)

['./temp/[7]/69d67e832c2b48cd92f4b04aed74c28d/model/study_best_trial.npy']

Found 1 best trials for subjects: [7]